In [8]:
AGENT_NAME = 'Models\ATLA\PPO agent 100 alts over 1000+200 2-3-21.zip'
ADVERSARY_PATH = 'Models\Adversary\default_adversary_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_default_rwd_480'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
ATK_NAME = 'Optimal myBB no init'
SAVE_DIR = "PPO agent 100 alts over 1000+200 2-3-21 results" + '/'


In [9]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from KBMproject.mybb import BrendelBethgeAttack as BBA

import pandas as pd
import numpy as np
import os

import KBMproject.utilities as utils

%matplotlib inline

Define RL agent

In [10]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from 'c:\\Users\\Broda-Milian\\anaconda3\\envs\\CityLearnART\\lib\\site-packages\\cloudpickle\\cloudpickle.py'>
  warnings.warn(
c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from 'c:\\Users\\Broda-Milian\\anaconda3\\envs\\CityLearnART\\lib\\site-packages\\cloudpickle\\cloudpickle.py'>
  warnings.warn(


In [11]:
adversary = PPO.load(path=ADVERSARY_PATH)
print('Model loaded from storage')

Model loaded from storage


In [12]:
env = utils.make_discrete_env(schema=DataSet.get_schema(DATASET_NAME),  
                        action_bins=agent.action_space[0].n,
                        seed=42)
cols = env.observation_names

In [13]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

Below we are making a dict which contains a 2d array of samples corresponding to each action, so the attack can select a sample of the target class closest to the clean input as an initialization.

After a few tens of iterations this had an ASR of 0.9, uncertain why

Below, for each action we select the sample with the highest logit softmax/confidence

In [18]:
%%time
import logging
logging.basicConfig(level=logging.WARN)
kwargs = dict(norm=np.inf,
        targeted=True, #default false
        overshoot= 1.1,
        steps=1000,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=1000, 
        )

kpi, obs, adv_obs, actions, targets, asr = utils.eval_targeted_attack(agent, adversary, 
                                                                        env=env, 
                                                                        ART_atk=utils.define_attack(agent, BBA, ART_kwargs=kwargs),
                                                                        mask=observation_masks,
                                                                        time_steps=None,
                                                                        )


100%|██████████| 8759/8759 [9:40:03<00:00,  3.97s/it, ASR=0.934]


CPU times: total: 9h 39min 40s
Wall time: 9h 40min 4s


In [19]:
display(kpi)

cost_function
annual_peak_average                       1.727378
carbon_emissions_total                    3.082044
cost_total                                2.840735
daily_one_minus_load_factor_average       0.963453
daily_peak_average                        2.943232
electricity_consumption_total             3.127384
monthly_one_minus_load_factor_average     1.003972
ramping_average                          16.387788
zero_net_energy                           1.698403
Name: District, dtype: float64

In [20]:
np.savetxt(f"{SAVE_DIR + ATK_NAME} targets.csv", targets, delimiter=",")
np.savetxt(f"{SAVE_DIR + ATK_NAME} actions.csv", actions, delimiter=",")

In [21]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/KPIs.csv updated


In [22]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_obs.csv')

In [23]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_adv_obs.csv')

In [24]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asr = pd.read_csv(asr_savename, index_col=0)
    df_asr[ATK_NAME] = asr
    df_asr.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    pd.DataFrame({ATK_NAME:asr}, index=['ASR']).to_csv(asr_savename)
    print(f'{asr_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/ASRs.csv updated
